In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
# openai work
from openai import OpenAI

client = OpenAI()
def stream_openai(client, role, message, model="gpt-4o-mini"):
    stream = client.chat.completions.create(
        model=model,
        messages=[{"role": role, "content": message}],
        stream=True,
    )
    message = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            message += chunk.choices[0].delta.content
    return message

In [9]:
# Preprocessor test
from pathlib import Path

from ragna.source_storages import RagnaDemoSourceStorage
from ragna.source_storages import LanceDB
from ragna.core import (
    LocalDocument, Document
)


storage = LanceDB()

temporal_doc_paths = [p for p in Path.cwd().joinpath("temporal_docs").iterdir() if p.suffix == ".md"]
temporal_docs = [
    LocalDocument.from_path(path) for path in temporal_doc_paths
]
storage.store("temporal_docs", temporal_docs)

/home/andrew/micromamba/envs/ragna-dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Preprocessor that rewords the question with context

## Without any context preprocessing

In [10]:
from typing import Optional

from ragna.assistants import RagnaDemoAssistant

from ragna import Rag
from ragna.assistants import Gpt4


chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="temporal_docs",
)
_ = await chat.prepare()
print(await chat.answer("What happened last year?"))

Last year, a rocket was built.


# With context preprocessing

To build on this, you could add more context like location

In [11]:
from typing import Optional
from datetime import datetime
from ragna.core import (
QueryPreprocessor, MetadataFilter, ProcessedQuery
)

base_prompt = (
"""You are a llm agent that is responsible for rewriting an input prompt for a RAG application. 
A question might contain hidden context that may not be recognized by the embedding model. 
An embedding of a question might not generate a close match to an embedding of a statement
that contains the answer to the question. Although this is technically something that should
be solved on the embedding model side, it is usually solved by rephrasing the question
before using it to retrieve sources. For example a question like "What happened last month?" 
likely won't get any close matches. Rephrasing the prompt to "What happened in December 2021?"
given that the question is asked January 2022. Things that may be important to consider when 
reworking the prompt would be the current context, that may that is not explicitly asked in the 
question but can be inferred, for instance the current date. 

current date: {}


Please reword the following prompt 
and only return the reworded prompt. I do not need to know your reasoning:


""".format(datetime.today()))

class TestPreprocessor(QueryPreprocessor):

    def __init__(self):
        # self.storage=storage
        # self.assistant=assistant
        self.messages = []

    def ask_assistant(self, prompt):
        instruction = (base_prompt + prompt)
        print(instruction)
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        processed_query = self.ask_assistant(query)
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [12]:
preprocessor = TestPreprocessor()
processed_query = preprocessor.process("What happened last year?", None)

You are a llm agent that is responsible for rewriting an input prompt for a RAG application. 
A question might contain hidden context that may not be recognized by the embedding model. 
An embedding of a question might not generate a close match to an embedding of a statement
that contains the answer to the question. Although this is technically something that should
be solved on the embedding model side, it is usually solved by rephrasing the question
before using it to retrieve sources. For example a question like "What happened last month?" 
likely won't get any close matches. Rephrasing the prompt to "What happened in December 2021?"
given that the question is asked January 2022. Things that may be important to consider when 
reworking the prompt would be the current context, that may that is not explicitly asked in the 
question but can be inferred, for instance the current date. 

current date: 2025-01-29 23:51:47.609820


Please reword the following prompt 
and only return the r

In [13]:
print(processed_query.processed_query)

What significant events occurred in 2024?


In [14]:
chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="temporal_docs",
)
_ = await chat.prepare()
print(await chat.answer(processed_query.processed_query))

In 2024, the significant event that occurred was a trip to the moon.


## Findings

This preprocesser can be effective for question where context is important. In an "agentic" workflow you may have a step that would determine and lookup the relevant context instead of always giving same context features. For example in the above prompt you could have a step to determine whether date is actually relevant to the question and only add it to the prompt if it is.

Note:
My prompt above may be too leading. Giving it a date example and then asking it a date relevant question may not translate to a question where something like location is important.

# Query Expansion

In [15]:
# Ragna docs
from ragna.source_storages import LanceDB

docs_path = Path.cwd().joinpath("../docs")

md_files = list(docs_path.glob("**/*.md"))
print(md_files[:2])

storage = LanceDB()

documents = [
            (
                document
                if isinstance(document, Document)
                else LocalDocument.from_path(document)
            )
            for document in md_files
        ]
storage.store("ragna_docs", documents)

[PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/preprocessing_demo/../docs/install.md'), PosixPath('/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/preprocessing_demo/../docs/index.md')]


In [21]:
#example_question = "how do I setup locally to contribute?"
example_question = "how do I build a rag application using ragna?"

In [22]:
chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="ragna_docs",
)
_ = await chat.prepare()
print(await chat.answer(example_question))

I'm sorry, but the information provided does not contain any details on how to build a RAG application using Ragna.


## Preprocessor

This preprocesser will first ask the llm to give a naive answer without any source materials it will then use the naive answer to query the sources which it will then prepend to the original prompt. 

In [23]:
from typing import Optional
from ragna.core import (
QueryPreprocessor, MetadataFilter, ProcessedQuery
)

base_prompt = (
"""You are a helpful software engineer. 
 how would you answer the following question from a more junior developer?

 question: {}
""")

class QueryExpansionPreprocessor(QueryPreprocessor):

    def __init__(self, storage):
        self.storage=storage
        # self.assistant=assistant
        self.messages = []

    def ask_assistant(self, prompt):
        instruction = (base_prompt.format(prompt))
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        hypothetical_answer = self.ask_assistant(query)
        print(hypothetical_answer)
        sources = self.storage.retrieve("ragna_docs", metadata_filter, hypothetical_answer)
        
        processed_query = "\n".join(s.content for s in sources) + "\n\n" + query
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [24]:
preprocessor = QueryExpansionPreprocessor(storage)

preprocessed_query = preprocessor.process(example_question, None)


To help a junior developer build a RAG (Retrieval-Augmented Generation) application using Ragna, I would first clarify that Ragna is likely a framework or library that supports the creation of RAG applications. Assuming Ragna is a fictional or hypothetical tool that we’re discussing as part of this learning exercise, here's how I would approach answering their question:

---

### Building a RAG Application Using Ragna

**1. Understand RAG Concept:**
Before diving into the implementation, it’s important to understand what a RAG application is. A RAG combines retrieval and generation components to enhance responses or outputs. Typically, this involves pulling in relevant information from a knowledge base (retrieval) and generating human-like textual responses based on that information (generation).

**2. Setting Up Your Environment:**
Make sure you have the following prerequisites installed:
- Python (version that is compatible with Ragna)
- Necessary dependencies (install via pip, e.g.,

In [25]:
print(preprocessed_query.processed_query)

# Frequently asked questions

## Why should I use Ragna and not X?

!!! tip "TL;DR"

    Ragna is the only tool out there that specializes in orchestrating RAG use cases
    with arbitrary components, as well as offering a Python API, a REST API, and a web
    UI for that.

!!! note

    Although we try to be objective as possible, this section is inheritly biased. If
    you are the author of a package we don't mention below but think we should or your
    package is mentiond but you feel we have mischaracterized it, please
    [get in touch](https://github.com/Quansight/ragna/discussions).

After the emergence of ChatGPT in November of 2022, the field of LLMs exploded. Today,
there are many providers for LLM REST APIs out there. With these, we also have a
plethora of Python packages to build applications around the provided APIs. We cannot
summarize the whole field here, so we stick to large projects in the Python ecosystem
for comparison.

| library or application                   

In [26]:
chat = Rag().chat(
    input=None,
    source_storage = storage,
    #assistant=RagnaDemoAssistant,
    assistant=Gpt4,
    corpus_name="ragna_docs",
)
_ = await chat.prepare()
print(await chat.answer(preprocessed_query.processed_query))

I'm sorry, but the provided sources do not contain information on how to build a RAG application using Ragna.


## Findings:

For this to be valuable I think you would need to be asking a question that is related but perhaps indirectly to the source material. It also may be unreliable. When I ran this code on 1/24/25, a got a response attempting the question, and I continued to get a response across tries. When I tried again 1/27/25 I got "I'm sorry, but the provided sources do not contain information on how to build a RAG application." To the best of my knowledge there were no changes and the processed query was the same in both instances.

response from 1/24/25:

```
To build a RAG (Retrieval-Augmented Generation) application using Ragna, you would need to follow these steps:

1. Install Ragna: You can install Ragna using pip, a package installer for Python. You can do this by running the command `pip install ragna` in your terminal.

2. Set up your configuration: Ragna uses a configuration file to set up the environment. This includes setting up the local root directory for storing files, the authentication class for user authentication, the key-value store class for temporary storage, and the document class for uploading and reading documents. You can also set up the source storages and assistants available for the user to use.

3. Write your RAG components: You would need to write your own RAG components, which include the document, source storage, and assistant classes. These classes should inherit from the respective base classes in Ragna and implement the required methods.

4. Run your application: Once you have set up your configuration and written your RAG components, you can run your application. If you are using the REST API, you would need to set up the hostname, port, and root path for the API.

Please note that this is a high-level overview of the process. The exact steps
```

# Query Expansion with ReRanking

The next preprocessor works the same as the above except that it also ranks similarity of the question to the sources that were found off of a retrieval from the naive answer. This could be useful when you want to expand the query but not too much.

In [28]:
from typing import Optional
from ragna.core import (
QueryPreprocessor, MetadataFilter, ProcessedQuery
)
from sentence_transformers import CrossEncoder

base_prompt = (
"""You are a helpful software engineer. Hypothetically given a specific software package in python,
 how would you answer the following question from a more junior developer?

 question:
""")

class QueryExpansionReRankingPreprocessor(QueryPreprocessor):

    def __init__(self, storage):
        self.storage=storage
        # self.assistant=assistant
        self.messages = []
        self.cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

    def ask_assistant(self, prompt):
        instruction = (base_prompt + prompt)
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer
    
    def rank_sources(self, sources, hypothetical_answer):
        return sources

    def cross_encoder_rerank(self, query, sources):
        #cross encoder reranker

        # Extract text content from Document objects and convert to strings
        document_texts = [doc.content for doc in sources]
        # Create pairs as strings
        pairs = [[query, doc_text] for doc_text in document_texts]
        # Predict scores for pairs
        scores = self.cross_encoder.predict(pairs)
        docs_and_scores = list(zip(scores, document_texts))
        sorted_ds = sorted(docs_and_scores, key=lambda x: x[0], reverse=True)
        top_docs = [x[1] for x in sorted_ds[:2]]
        return top_docs

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        hypothetical_answer = self.ask_assistant(query)
        sources = self.storage.retrieve("ragna_docs", metadata_filter, hypothetical_answer)
        top_docs = self.cross_encoder_rerank(query, sources)

        
        processed_query = "\n".join(s for s in top_docs) + "\n\n" + query
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [29]:
preprocessor = QueryExpansionReRankingPreprocessor(storage)

preprocessed_query = preprocessor.process(example_question, None)
preprocessed_query

/home/andrew/micromamba/envs/ragna-dev/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /home/conda/feedstock_root/build_artifacts/libtorch_1737846236919/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


ProcessedQuery(original_query='how do I build a rag application using ragna?', processed_query='# Frequently asked questions\n\n## Why should I use Ragna and not X?\n\n!!! tip "TL;DR"\n\n    Ragna is the only tool out there that specializes in orchestrating RAG use cases\n    with arbitrary components, as well as offering a Python API, a REST API, and a web\n    UI for that.\n\n!!! note\n\n    Although we try to be objective as possible, this section is inheritly biased. If\n    you are the author of a package we don\'t mention below but think we should or your\n    package is mentiond but you feel we have mischaracterized it, please\n    [get in touch](https://github.com/Quansight/ragna/discussions).\n\nAfter the emergence of ChatGPT in November of 2022, the field of LLMs exploded. Today,\nthere are many providers for LLM REST APIs out there. With these, we also have a\nplethora of Python packages to build applications around the provided APIs. We cannot\nsummarize the whole field her

## Findings:

I think I need a more extensive dataset and appropriate question. This seems like it could be useful in the case where you need to expand the query, but you have an extensive dataset so don't want to expand it by too much. I am also concerned that since you are scoring their similarity to the question and not the naive answer that you may not actually be scoring on the important expanded similarity.



# More experiments

Below is an attempt to have an llm come up the with the steps necessary to find relevant documents. 

In [64]:
from typing import Optional
from ragna.core import (
QueryPreprocessor, MetadataFilter, ProcessedQuery
)
from sentence_transformers import CrossEncoder

base_prompt = (
"""Given the question below. Make a plan on how best to answer the question.
 
retrieve relevant sources from a vector database to best answer the question and only return the plan.

 question:
""")

class QueryExpansionReRankingPreprocessor(QueryPreprocessor):

    def __init__(self, storage):
        self.storage=storage
        # self.assistant=assistant
        self.messages = []
        self.cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

    def ask_assistant(self, prompt):
        instruction = (base_prompt + prompt)
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        hypothetical_answer = self.ask_assistant(query)
        print(hypothetical_answer)
        sources = self.storage.retrieve("ragna_docs", metadata_filter, hypothetical_answer)


        
        processed_query = "\n".join(s.content for s in sources) + "\n\n" + query
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [65]:
preprocessor = QueryExpansionReRankingPreprocessor(storage)

preprocessed_query = preprocessor.process(example_question, None)
preprocessed_query.processed_query

### Plan to Retrieve Relevant Sources from a Vector Database for Building a RAG Application

1. **Define RAG Application Criteria:**
   - Identify key topics and subtopics related to “RAG application” (e.g., architecture, frameworks, data sources, models).
   - Establish keywords and phrases (e.g., "retrieval-augmented generation", "building RAG applications", "RAG model implementation").

2. **Prepare Vector Database Query:**
   - Formulate vector embeddings for identified keywords using a pre-trained language model.
   - Create a combination of queries that encompass both general concepts and specific technical aspects.

3. **Vector Search Execution:**
   - Utilize the vector search capabilities of the database to retrieve documents that closely match the embeddings created.
   - Set parameters for the search (e.g., similarity thresholds, number of results).

4. **Filter Retrieved Sources:**
   - Review the returned results for relevance based on predefined criteria (e.g., publicatio

'# Frequently asked questions\n\n## Why should I use Ragna and not X?\n\n!!! tip "TL;DR"\n\n    Ragna is the only tool out there that specializes in orchestrating RAG use cases\n    with arbitrary components, as well as offering a Python API, a REST API, and a web\n    UI for that.\n\n!!! note\n\n    Although we try to be objective as possible, this section is inheritly biased. If\n    you are the author of a package we don\'t mention below but think we should or your\n    package is mentiond but you feel we have mischaracterized it, please\n    [get in touch](https://github.com/Quansight/ragna/discussions).\n\nAfter the emergence of ChatGPT in November of 2022, the field of LLMs exploded. Today,\nthere are many providers for LLM REST APIs out there. With these, we also have a\nplethora of Python packages to build applications around the provided APIs. We cannot\nsummarize the whole field here, so we stick to large projects in the Python ecosystem\nfor comparison.\n\n| library or appli

In [66]:
base_prompt = ("""how would I break up the following steps to use in a python program:

steps:
""")
prompt = preprocessed_query.processed_query
instruction = (base_prompt + prompt)
assistant_answer = stream_openai(client, "user", instruction)
assistant_answer

'To break down the steps provided in your text for use in a Python program, first, we should identify the key components that will help shape the structure of the application. Since the steps deal with FAQs about Ragna and its comparison with similar libraries/applications, we can organize them into functions or data structures to facilitate user queries or interactions.\n\nHere\'s how you could structure your Python program:\n\n1. **Create a FAQ Class**: This class will contain the FAQs and methods to retrieve them.\n\n2. **Implement Comparison Table**: This can be either a method in the same class or a separate data structure that compares Ragna with other libraries.\n\n3. **Define Methods for User Interaction**: Allow users to query information about Ragna and the comparison.\n\nHere’s an example implementation:\n\n```python\nclass RagnaFAQ:\n    def __init__(self):\n        self.faqs = {\n            "Why should I use Ragna and not X?": {\n                "TL;DR": "Ragna is the onl

To break down the steps provided in your text for use in a Python program, first, we should identify the key components that will help shape the structure of the application. Since the steps deal with FAQs about Ragna and its comparison with similar libraries/applications, we can organize them into functions or data structures to facilitate user queries or interactions.

Here's how you could structure your Python program:

1. **Create a FAQ Class**: This class will contain the FAQs and methods to retrieve them.

2. **Implement Comparison Table**: This can be either a method in the same class or a separate data structure that compares Ragna with other libraries.

3. **Define Methods for User Interaction**: Allow users to query information about Ragna and the comparison.

Here’s an example implementation:

```python
class RagnaFAQ:
    def __init__(self):
        self.faqs = {
            "Why should I use Ragna and not X?": {
                "TL;DR": "Ragna is the only tool out there th

## Create metadata filter

Below is an attempt to create metadata filters themselves with an llm.

In [85]:
base_prompt = (
"""
question: What happened last year?

This question will be used to retrieve relevant sources from a vector database.
The vector database contains sources with attached metadata. Below is an example 
filters that can be used to more effectively retrieve data from the vector database.
For the above question. Return a list of relevant metadata filters.

[
    MetadataFilter.raw("raw"),
    MetadataFilter.and_(
        [
            MetadataFilter.raw("raw"),
            MetadataFilter.eq("key", "value"),
        ]
    ),
    
    MetadataFilter.or_(
        [
            MetadataFilter.raw("raw"),
            MetadataFilter.eq("key", "value"),
        ]
    ),
    MetadataFilter.and_(
        [
            MetadataFilter.raw("raw"),
            MetadataFilter.or_(
                [
                    MetadataFilter.eq("key", "value"),
                    MetadataFilter.ne("other_key", "other_value"),
                ]
            ),
        ]
    ),
    MetadataFilter.or_(
        [
            MetadataFilter.raw("raw"),
            MetadataFilter.and_(
                [
                    MetadataFilter.eq("key", "value"),
                    MetadataFilter.ne("other_key", "other_value"),
                ]
            ),
        ]
    ),
    MetadataFilter.eq("key", "value"),
    MetadataFilter.ne("key", "value"),
    MetadataFilter.lt("key", 1),
    MetadataFilter.le("key", 0),
    MetadataFilter.gt("key", 1),
    MetadataFilter.ge("key", 0),
    MetadataFilter.in_("key", ["foo", "bar"]),
    MetadataFilter.not_in("key", ["foo", "bar"]),
]   

The possible metadata for the relevant sources are as follows:
"""
)

class MetadataFilterPreprocessor(QueryPreprocessor):

    def __init__(self, storage, corpus_name):
        self.storage=storage
        self.corpus_name = "ragna_docs"
        # self.assistant=assistant
        self.messages = []

    def ask_assistant(self, prompt):

        instruction = (base_prompt + prompt)
        assistant_answer = stream_openai(client, "user", instruction)
        return assistant_answer

    def process(self, query: str, metadata_filter: Optional[MetadataFilter]):
        prompt = self.storage.list_metadata(self.corpus_name)
        metadata = str(prompt[self.corpus_name])
        metadata_filters = self.ask_assistant(metadata)
        print(metadata_filters)
        #sources = self.storage.retrieve("ragna_docs", metadata_filter, hypothetical_answer)

        sources = ""
        processed_query = "\n".join(s.content for s in sources) + "\n\n" + query
        return ProcessedQuery(
            original_query=query,
            processed_query=processed_query,
            metadata_filter=None,
            processor_name=self.display_name()
        )

In [86]:
MetadataFilterPreprocessor(storage, "ragna_docs").process("What happened last year?", None)

To effectively retrieve relevant sources from the vector database for the question about what happened last year, we can apply the following metadata filters:

```python
[
    MetadataFilter.eq("year", 2022),  # Assuming we're looking for events from the previous year
    MetadataFilter.eq("document_name", "release-notes.md"),  # Assuming release notes may contain relevant yearly summaries
    MetadataFilter.eq("key", "year"),  # Filter that could relate to a specific key about events
    MetadataFilter.or_(
        [
            MetadataFilter.eq("key", "event"), # This could refer to specific events reported in 2022
            MetadataFilter.eq("key", "news"),  # This could refer to news summaries from 2022
        ]
    ),
    MetadataFilter.and_(
        [
            MetadataFilter.gt("size", 0),  # Ensures that we are not retrieving empty documents
            MetadataFilter.eq("extension", ".md"),  # Filter to only include markdown documents
        ]
    ),
    MetadataFilter.

ProcessedQuery(original_query='What happened last year?', processed_query='\n\nWhat happened last year?', metadata_filter=None, processing_history=[])

In [73]:
for document in documents:
    print(document.metadata)

{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/install.md', 'extension': '.md', 'size': 1304}
{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/index.md', 'extension': '.md', 'size': 1866}
{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/community/welcome.md', 'extension': '.md', 'size': 1469}
{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/community/contribute.md', 'extension': '.md', 'size': 3623}
{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/explanations/what-is-rag.md', 'extension': '.md', 'size': 136}
{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/examples/README.md', 'extension': '.md', 'size': 12}
{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/references/python-api.md', 'extension': '.md', 'size': 128}
{'path': '/home/andrew/.dropbox-hm/Dropbox/quansight/dev/ragna/ragna/docs/references/deploy.md', 

In [82]:
str(storage.list_metadata("ragna_docs")["ragna_docs"])

"{'document_id': ('str', ['0424ae53-ae02-46c8-9b0f-5c416fac0472', '09d71a8a-c807-488b-bbb8-be731b880086', '0f72b676-9b07-4d22-b983-615b6fe084ba', '1d159e07-a0ec-4c6f-8bf4-0917725afc3e', '1e4a4428-e7e9-405f-9b6f-c40d2bde31c2', '24cc63bc-69b1-4b3b-bf2c-bf11e867a689', '25e7b0f5-9a2f-405f-9df3-c8f4e6d68ff0', '264171f2-2785-4a52-b223-19e525813290', '26eb3473-2db4-42cd-bc7f-64fe968f1b6b', '2a4ef41f-c049-4f98-9d0e-4c83a488edb4', '3e38190c-85de-4871-94e3-45a5c6f890e7', '4046f5fe-4f0c-4812-a8a3-ebf2598d2803', '43db5fee-cb9e-4f4f-8fca-b08b00127295', '5d26e37e-2434-490f-889c-b80492619836', '6d8efa94-5ad8-476d-bdf8-39f46e3b6117', '71f432ec-e2a0-47d5-b0a7-f873d5083e7c', '72fd4d34-d42f-481b-9b76-0a12b14f5cab', '73bc7d98-3c2d-466b-b8c3-ccff46a98892', '7c87c95a-aba4-4199-9648-69e2459c8d02', '7d50b63c-d2ef-4990-8ec6-3de124904853', '82b1ad33-3d9f-4f61-b1af-ab56e8b94bf5', '8a7278c7-7f1b-48c8-9948-36a3b6aae524', '8b01fc99-c6cb-469f-8371-f3738d51e4fa', '9c4bfdb7-8ee2-42ee-ab8c-b068bdc2cc61', 'a4c3753a-8bd3